In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import sys
sns.set_style('darkgrid')
%matplotlib qt

In [ ]:
"""
Computing the velovity averaged crossection with pauli blocking taken into account

# We will consider the collision of two identical particles (nn -> nn) in COM of frame
# Assume the fermi momentum is normalized to p_f = 1
# Generating random momentum from the interval (0, 1) with pdf f(p) = 3p^2 (momentum near the fermi surface are more prefered). This can be reframed as
p^3 having uniform distribution

"""

In [ ]:
no_of_events = 10**5

# Random distribution for the initial and final momentum
p1 = ( np.random.uniform(0, 1, no_of_events) ) ** (1/3)
p2 = ( np.random.uniform(0, 1, no_of_events) ) ** (1/3)

# Random distribution for cosine of two arbitrary angles during the collision

calpha = np.random.uniform(-1, 1, no_of_events)   # angle between p1 and p2 in lab frame
ctheta = np.random.uniform(-1, 1, no_of_events)   # angle between p3 and p1 + p2 in com frame

pf_n = []  # final momentum of neutron
pf_nd = [] # final momentum of n'
p_com = [] # magnitude of momentum transfer

for i in range(no_of_events):
    pf_n.append( (p1[i]**2 + p2[i]**2 + np.sqrt( (p1[i]**2 + p2[i]**2)**2  - 4*(p1[i]*p2[i]*calpha[i])**2 )*ctheta[i])**(1/2) / np.sqrt(2) )
    pf_nd.append( (p1[i]**2 + p2[i]**2 - np.sqrt( (p1[i]**2 + p2[i]**2)**2  - 4*(p1[i]*p2[i]*calpha[i])**2 )*ctheta[i])**(1/2) / np.sqrt(2) )
    p_com.append( np.sqrt(p1[i]**2 + p2[i]**2 - 2 * p1[i] * p2[i] * calpha[i]) )

# Now, we perform the sampling only over the states where the final momentum of the neutron is higher than the fermi momentum

allowed_pf_n = []
allowed_pf_nd = []
allowed_p_com = []

for i in range(no_of_events):
    if pf_n[i] >= 1 :
        allowed_pf_n.append(pf_n[i])
        allowed_pf_nd.append(pf_nd[i])
        allowed_p_com.append(p_com[i])


In [ ]:
# Sampling for the neutron and proton collision

p2_p = np.sqrt(0.29) * ( np.random.uniform(0, 1, no_of_events) ) ** (1/3)

pf_p = []
pf_pnd = []
p_pcom = []

for i in range(no_of_events):
    pf_p.append( (p1[i]**2 + p2_p[i]**2 + np.sqrt( (p1[i]**2 + p2_p[i]**2)**2  - 4*(p1[i]*p2_p[i]*calpha[i])**2 )*ctheta[i])**(1/2) / np.sqrt(2) ) # final momentum of proton
    pf_pnd.append( (p1[i]**2 + p2_p[i]**2 - np.sqrt( (p1[i]**2 + p2_p[i]**2)**2  - 4*(p1[i]*p2_p[i]*calpha[i])**2 )*ctheta[i])**(1/2) / np.sqrt(2) ) # final momentum of n'
    p_pcom.append( np.sqrt(p1[i]**2 + p2_p[i]**2 - 2 * p1[i] * p2_p[i] * calpha[i]) ) # Momentum transfers

allowed_pf_p = []
allowed_pf_pnd = []
allowed_p_pcom = []


# Allowed events with final momentum of proton above the fermi surface
for i in range(no_of_events):
    if pf_p[i] >= np.sqrt(0.29) :
        allowed_pf_p.append(pf_p[i])
        allowed_pf_pnd.append(pf_pnd[i])
        allowed_p_pcom.append(p_pcom[i])

In [ ]:
"""  We will be using natural units hbar = c = 1 and the GeV is our only unit of measure"""

In [ ]:
import pandas as pd
import os

mun_nd_path = os.path.join('data','mun_from_nd.csv')
mun_nd = np.transpose(np.array(pd.read_csv(mun_nd_path, delimiter=',', header=None)))

mup_nd_path = os.path.join('data','mup_from_nd.csv')
mup_nd = np.transpose(np.array(pd.read_csv(mup_nd_path, delimiter=',', header=None)))

mun_path = os.path.join('data','mun.csv')
mun = np.transpose(np.array(pd.read_csv(mun_path, delimiter=',', header=None)))

mup_path = os.path.join('data','mup.csv')
mup = np.transpose(np.array(pd.read_csv(mup_path, delimiter=',', header=None)))

# Converting from mu_n vs r to p_f vs r (The data is available in units MeV vs km))
mass_of_neutron =  0.94 # (in GeV)

In [ ]:
# Defining the fermi momentum as a function of radius for neutrons

r_n = mun_nd[0] * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
pf_n = np.sqrt(2 * 940.6 * mun_nd[1]) * 10**(-3)  # (in GeV)

func_pf_n = scipy.interpolate.interp1d(r_n, pf_n, kind='cubic', fill_value=0, bounds_error=False)

# Defining the fermi momentum as a function of radius for protons

r_p = mup_nd[0] * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
pf_p = np.sqrt(2 * 940.6 * mup_nd[1]) * 10**(-3)  # (in GeV)

func_pf_p = scipy.interpolate.interp1d(r_p, pf_p, kind='cubic', fill_value=0, bounds_error=False)

# Defining the fermi energy accouting nuclear interaction for neutrons and protons

r_nm_n = mun[0] * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
mun_nm = mun[1] * 10**(-3) # (in GeV)

func_mun = scipy.interpolate.interp1d(r_nm_n, mun_nm, kind='cubic', fill_value=0, bounds_error=False)

r_nm_p = mup[0] * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
mup_nm = mup[1] * 10**(-3) # (in GeV)

func_mup = scipy.interpolate.interp1d(r_nm_p, mup_nm, kind='cubic', fill_value=0, bounds_error=False)

In [ ]:
# Phase shifts for the required reactions
NN_1S0_path = os.path.join('data', 'phaseshift-NN-1S0.csv')
NN_1S0 = np.transpose(np.array(pd.read_csv(NN_1S0_path, delimiter= ',', header=None, engine='python')))

NP_1S0_path = os.path.join('data', 'phaseshift-NP-1S0.csv')
NP_1S0 = np.transpose(np.array(pd.read_csv(NP_1S0_path, delimiter= ',', header=None, engine='python')))

NP_3P1_path = os.path.join('data', 'phaseshift-NP-3P1.csv')
NP_3P1 = np.transpose(np.array(pd.read_csv(NP_3P1_path, delimiter= ',', header=None, engine='python')))

# NN 1S0 phase
E_N0 = NN_1S0[0] # (in GeV)
P_N0 = NN_1S0[1] # (in Radians)

func_N0 = scipy.interpolate.interp1d(E_N0, P_N0, kind='cubic', fill_value=0, bounds_error=False)

# NP 1S0 phase
E_P0 = NP_1S0[0] # (in GeV)
P_P0 = NP_1S0[1] # (in Radians)

func_P0 = scipy.interpolate.interp1d(E_P0, P_P0, kind='cubic', fill_value=0, bounds_error=False)

# NP 3S1 phase
E_P1 = NP_3P1[0] # (in GeV)
P_P1 = NP_3P1[1] # (in Radians)

func_P1 = scipy.interpolate.interp1d(E_P1, P_P1, kind='cubic', fill_value=0, bounds_error=False)

In [ ]:
# Delta E vs r

DeltaEvsr_path = os.path.join('data', 'DeltaEvsr.dat')
DeltaEvsr = np.transpose(np.array(pd.read_csv(DeltaEvsr_path, delimiter= '\t', header=None, engine='python')))

r_DE = DeltaEvsr[0] * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
DeltaE = DeltaEvsr[1] * 10**(-3) # (in GeV)

func_DeltaE = scipy.interpolate.interp1d(r_DE, DeltaE, kind='cubic', fill_value=0, bounds_error=False)

In [ ]:
plt.plot(r_DE / (1000 / (1.9733 * 10**(-16))), func_DeltaE(r_DE))

In [ ]:
"""
Defining the functions that is going to give the total luminosity of the star
"""

In [ ]:
g_n = 2
g_p = 1
f_n = 1
f_p = 2


def Energy_rate(r, epsilon):

    """ Neutron mediated reaction """
    E_sigma_nn = 0
    avg_sigma_nn = 0
    n_n = len(allowed_p_com)
    
    for i in range(n_n):
        # Energy at which the event is happening
        energy_n_r = (allowed_p_com[i] * func_pf_n(r))**2 / (2 * mass_of_neutron)
        # The corresponding phase shift
        phase_n = np.sin(func_N0(energy_n_r))**2
        # Cross section for that event
        sigma_nn = (1/4) * 16 * np.pi * phase_n / (allowed_p_com[i] * func_pf_n(r))**2
        # Kinetic energy of the outgoing n'
        KEn_nd = (allowed_pf_nd[i] * func_pf_n(r)) ** 2 / (2 * mass_of_neutron)
        # Just sigma for that event
        avg_sigma_nn += sigma_nn / n_n
        # E * sigma * v for that event
        E_sigma_nn += (func_mun(r) - KEn_nd) * sigma_nn * (allowed_p_com[i] * func_pf_n(r)) / n_n


    # number density as computed from fermi energy
    numden_n = func_pf_n(r) ** 3 / (3 * np.pi**2)

    # Contribution corresponding to the nn -> nn reaction
    E_rate_n = f_n * numden_n * g_n * (epsilon/func_DeltaE(r))**2 * E_sigma_nn

    """ Proton mediated reaction """

    E_sigma_np = 0
    avg_sigma_np = 0
    n_p = len(allowed_p_pcom)

    for j in range(n_p):
        # Naive Fermi energy at that radius
        energy_p_r = (allowed_p_pcom[j] * func_pf_n(r))**2 / (2 * mass_of_neutron)

        phase_p = np.sin(func_P0(energy_p_r))**2 + 3 * np.sin(func_P1(energy_p_r))**2

        sigma_np = (1/4) * 16 * np.pi * phase_p / (allowed_p_pcom[j] * func_pf_n(r))**2  # normalization for multiplying with fermi momentum of neutron has already been done

        KEp_nd = (allowed_pf_pnd[j] * func_pf_n(r)) ** 2 / (2 * mass_of_neutron)

        avg_sigma_np += sigma_np / n_p

        E_sigma_np += (func_mun(r) - KEp_nd) * sigma_np * (allowed_p_pcom[j] * func_pf_n(r)) /n_p

    # number density of proton as computed from fermi energies
    numden_p = func_pf_p(r)**3 / (3 * np.pi **2)

    # Contribution corresponding to the np -> np reaction
    E_rate_p = f_p * numden_p * g_p * (epsilon/func_DeltaE(r))**2 * E_sigma_np

    E_rate = E_rate_n + E_rate_p

    return avg_sigma_nn, avg_sigma_np, E_rate


In [ ]:
R = np.linspace(0.05, 8.1, 100) * 1000 / (1.9733 * 10**(-16)) # (in GeV^-1)
result = Energy_rate(R, 1)

plt.plot(R / (1000 / (1.9733 * 10**(-16))),result[0])
plt.plot(R/ (1000 / (1.9733 * 10**(-16))), result[1])
plt.show()


In [ ]:

import scipy.integrate

def Total_luminosity(epsilon):

    def integrand(r):
        numden_n = func_pf_n(r) ** 3 / (3 * np.pi**2)
        return 4 * np.pi * r**2 * numden_n * Energy_rate(r, epsilon)[2]
    
    # we will perform the integration in the interval 0 to 12.6 km but in GeV^-1 units
    r_min = 0.04 * 1000 / (1.9733 * 10**(-16))
    r_max = 8.1 * 1000 / (1.9733 * 10**(-16))

    r = np.linspace(r_min, r_max,  2**5 + 1)

    f_r = integrand(r)
    
    return  scipy.integrate.romb(f_r, dx = r[1] - r[0])

In [ ]:
# Luminosity as computed from a 42000 Kelvin neutron star from 12.6 km
def lum_NS(T_NS):
    rate_neutron_star = 4 * np.pi * (8.1 * 10 ** 3) **2 * 5.67 * 10 ** (-8) * (T_NS) ** 4 * 6.242 * 10 ** 9 * 6.58 * 10 ** (-25) # In  GeV^2
    return rate_neutron_star

In [ ]:
epsilon = np.sqrt(lum_NS(30000) / Total_luminosity(1))*10**9

In [ ]:
print(f"Epsilon in eV = {epsilon}")